Use PCA to get linear combination of original features.
Then train the model with SVM with slack/gaussian kernel. 

In [ ]:
File Paths:  
- training data: data/gisette_train.data
- validation data: data/gisette_valid.data
- test data: data/gisette_test.data

In [2]:
import numpy as np
import cvxopt
import cvxopt.solvers

#read file and store data to X and Y 
def read_data(filename): 
    file = open(filename, 'r', encoding='utf-8-sig')
    dataset = []
    for line in file:
        data = line.split(',')
        y_data = int(data[0])
        x_data = [float(x) for x in data[1:]] 
        dataset.append((x_data, y_data))

    X = np.array([x for x, y in dataset])
    Y = np.array([y for x, y in dataset])

    return X, Y

#gaussian kernel equation k(x, z)
def g_kernel(x, z, s): 
    return np.exp(-np.linalg.norm(x-z)**2/(2*(s**2)))

#train data to get optimal lagrange multiplier
def train(c, s, X, Y): 
    num_of_data, _ = X.shape

    #Gaussian Matrix
    K_matrix = np.zeros((num_of_data, num_of_data))
    for i in range(num_of_data): 
        for j in range(num_of_data):
            K_matrix[i, j] = g_kernel(X[i], X[j], s)

    #P_matrix
    P_matrix = np.outer(Y, Y) * K_matrix

    #q_vector
    q_vector = -np.ones(num_of_data)

    #G_matrix
    G_matrix = np.identity(num_of_data)
    G_matrix = np.vstack((G_matrix, -np.identity(num_of_data)))

    #h_vector
    h_vector = np.ones(num_of_data) * c
    h_vector = np.hstack((h_vector, np.zeros(num_of_data)))

    P = cvxopt.matrix(P_matrix)
    q = cvxopt.matrix(q_vector)
    G = cvxopt.matrix(G_matrix)
    h = cvxopt.matrix(h_vector) 
    A = cvxopt.matrix(Y.reshape(1, num_of_data) * 1.) 
    b = cvxopt.matrix(0.0) 

    cvxopt.solvers.options['show_progress'] = False #disable progess to console 
    sol = cvxopt.solvers.qp(P, q, G, h, A, b) 
    lagrange_multiplier = np.array(sol['x'])
    
    #get support vectors 
    sv_indices = np.where(lagrange_multiplier > 1e-5)[0]
    sv_lagrange_multiplier = lagrange_multiplier[sv_indices]
    support_vectors = X[sv_indices]
    sv_y = Y[sv_indices]

    return sv_indices, sv_lagrange_multiplier, support_vectors, sv_y, K_matrix

#constuct matrix W
def construct_W(X): 
    #get transpose of X (column becomes the data point)
    trans_X = X.T
    num_data = trans_X.shape[1]

    #compute sum of all data points 
    sum = 0
    for j in range(num_data): 
        sum += trans_X[:, j]
    #compute mean
    mean = sum / num_data

    W = np.zeros(trans_X.shape)
    for i in range(num_data): 
        W[:, i] = trans_X[:, i] - mean
    
    return W

#compute smallest k for a given percentage 
def compute_smallest_k(scaled_eigenvalues, percentage): 
    k_sum = 0
    k = 0

    while (k_sum < percentage): 
        k_sum += scaled_eigenvalues[k]
        k += 1

    return k

#compute K
def compute_K(eigenvalues): 
    K = []
    #sum all eigenvalues
    sum = np.sum(eigenvalues)
    #divide each by the sum
    scaled_eigenvalues = eigenvalues / sum
    
    #k99
    k = compute_smallest_k(scaled_eigenvalues, 0.99)
    K.append(k)

    #k95
    k = compute_smallest_k(scaled_eigenvalues, 0.95)
    K.append(k)

    #k90
    k = compute_smallest_k(scaled_eigenvalues, 0.90)
    K.append(k)

    #k85
    k = compute_smallest_k(scaled_eigenvalues, 0.85)
    K.append(k)

    #k80
    k = compute_smallest_k(scaled_eigenvalues, 0.80)
    K.append(k)

    return np.array(K)

#main: 
#training dataset
train_X, train_Y = read_data('data/gisette_train.data')
#test dataset
X, Y = read_data("data/gisette_test.data")

#mean-centering the data
means = np.mean(train_X, axis=0)
mean_centered_train_X = train_X - means

means = np.mean(X, axis=0)
mean_centered_X = X - means

#PCA
W = construct_W(mean_centered_train_X)
covariance_matrix = np.dot(W, W.T)
eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

#sort eigenvalues/eigenvectors in decreasing order
sorted_indices = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[sorted_indices]
eigenvectors = eigenvectors[:, sorted_indices] #reorder the columns

K = compute_K(eigenvalues)
C = [1e6]
sigma = [1e4]  

#compute k-dimensional subspace for each k
for k in K: 
    #get first k eigenvectors 
    k_eigenvectors = eigenvectors[:, :k]
    projected_train_x = np.dot(mean_centered_train_X, k_eigenvectors)
    projected_x = np.dot(mean_centered_X, k_eigenvectors)

    for c in C: 
        for s in sigma: 
            correct_predictions = 0
            total_data = projected_x.shape[0]
            sv_indices, sv_lagrange_multiplier, support_vectors, sv_y, K_matrix = train(c, s, projected_train_x, train_Y)
            sv_y = sv_y.reshape(sv_y.shape[0], 1)

            #compute b
            b = 0.0
            for i in range(len(sv_indices)):
                b += sv_y[i] 
                b -= np.sum(sv_lagrange_multiplier * sv_y * K_matrix[sv_indices, :][i])
            b /= len(sv_indices)

            #prediction
            for x, y in zip(projected_x, Y):
                sum = 0
                for i in range(len(sv_indices)):
                    sum += sv_lagrange_multiplier[i] * sv_y[i] * g_kernel(support_vectors[i], x, s)
                f_x = sum + b
        
                result = y * f_x
                if (result > 0): #correct classification 
                    correct_predictions += 1
                
            accuracy = (correct_predictions / total_data) * 100
            print('k:', k)
            print("c:", c)
            print("sigma:", s)
            print("accuracy:", accuracy, "%")

[2695 1761 1320 1051  855]


KeyboardInterrupt: 